import sys
import os
import matplotlib.pyplot as plt

# ---------------------------------------------------------------------------
# Projektpfad setzen, damit "model" importierbar ist
# ---------------------------------------------------------------------------
project_root = os.path.abspath("..")
print("Project root:", project_root)

if project_root not in sys.path:
    sys.path.append(project_root)

print("sys.path enthält project_root:", project_root in sys.path)

from model.simulation import load_scenario, simulate_load_profile

# ---------------------------------------------------------------------------
# Szenario laden und Simulation durchführen
# ---------------------------------------------------------------------------
scenario_name = "office"            # gewünschtes Szenario
scenario_path = f"../scenarios/{scenario_name}.yaml"
print("Verwendetes Szenario:", scenario_path)

scenario = load_scenario(scenario_path)
timestamps, load_kw, sessions = simulate_load_profile(scenario)

# ---------------------------------------------------------------------------
# Kontrollen: Zeitbereich und Simulationshorizont
# ---------------------------------------------------------------------------
print(f"Erster Timestamp: {timestamps[0]}")
print(f"Letzter Timestamp: {timestamps[-1]}")

time_resolution_min = scenario["time_resolution_min"]
steps_per_day = int(round(24 * 60 / time_resolution_min))
simulated_days = len(timestamps) / steps_per_day

print(f"Simulierte Tage (aus Zeitreihe berechnet): {simulated_days}")
print(f"simulation_horizon_days laut YAML:       {scenario['simulation_horizon_days']}")

# ---------------------------------------------------------------------------
# Lastprofil als Diagramm
# ---------------------------------------------------------------------------
plt.figure(figsize=(12, 4))
plt.plot(timestamps, load_kw)
plt.xlabel("Zeit")
plt.ylabel("Leistung [kW]")
plt.title(f"Lastprofil – Szenario '{scenario_name}'")
plt.grid(True)
plt.tight_layout()
plt.gcf().autofmt_xdate()   # Datum auf der x-Achse lesbar machen
plt.show()